# HyDE Table Retrieval

Hypothetical Document Embeddings for table search:
1. Generate table descriptions using LLM
2. Generate hypothetical table descriptions from queries using LLM
3. Encode and retrieve using MiniLM-L6-v2 + FAISS

## Setup

In [ ]:
import pandas as pd
import numpy as np
import json
import faiss
from sentence_transformers import SentenceTransformer
from collections import defaultdict
from tqdm import tqdm

from openai import OpenAI
from functools import reduce
import time

client = OpenAI(api_key="<insert key here>")


/opt/anaconda3/envs/smartcat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Data

In [3]:
# Load tables
tables_df = pd.read_csv('data/wikitables_mini.csv')
print(f"Loaded {len(tables_df)} tables")
print(f"Columns: {list(tables_df.columns)}")
tables_df.head(2)

Loaded 2932 tables
Columns: ['table_id', 'page_title', 'section_title', 'table_caption', 'headers', 'sample_data']


,table_id,page_title,section_title,table_caption,headers,sample_data
0,table-0001-249,Auburn Tigers swimming and diving,Summer Olympic Games Beijing 2008,Summer Olympic Games Beijing 2008,"[""Athlete"", ""Nation"", ""Total"", ""Gold"", ""Silver...","[[""[Fr\u00e9d\u00e9rick_Bousquet|Fr\u00e9d\u00..."
1,table-0001-400,Bisphenol A,Low-dose exposure in animals,Low-dose exposure in animals,"[""Dose (\u00b5g/kg/day)"", ""[Environmental_Work...","[[""0.025"", ""\""Permanent changes to genital tra..."


## Generate Table Descriptions (HyDE)

Use LLM to generate natural descriptions from table metadata.

In [ ]:
def generate_table_description_with_llm(row, max_chars=256):
    """
    Generate table description using LLM.
    
    TODO: Replace this with actual LLM call.
    """
    
    # Prepare table metadata as prompt context
    metadata = []
    if pd.notna(row['table_caption']):
        metadata.append(f"Caption: {row['table_caption']}")
    if pd.notna(row['page_title']):
        metadata.append(f"Page: {row['page_title']}")
    if pd.notna(row['section_title']):
        metadata.append(f"Section: {row['section_title']}")
    
    try:
        headers = json.loads(row['headers'])
        if headers:
            metadata.append(f"Columns: {', '.join([str(h) for h in headers[:10]])}")
    except:
        pass
    
    try:
        sample_data = json.loads(row['sample_data'])
        if sample_data and len(sample_data) > 0:
            sample_str = str(sample_data[0][:5])
            metadata.append(f"Sample: {sample_str}")
    except:
        pass
    
    metadata_str = '\n'.join(metadata)
        
    # Fallback: concatenate metadata (replace with actual LLM response)
    description = ' '.join(metadata)
    
    return description[:max_chars]

# Test
print("Example LLM-generated descriptions:")
print("=" * 80)
for i in range(3):
    desc = generate_table_description_with_llm(tables_df.iloc[i])
    print(f"\n{i+1}. {desc}")

Example LLM-generated descriptions:

1. Caption: Summer Olympic Games Beijing 2008 Page: Auburn Tigers swimming and diving Section: Summer Olympic Games Beijing 2008 Columns: Athlete, Nation, Total, Gold, Silver, Bronze, Events Sample: ['[Frédérick_Bousquet|Frédérick Bousquet]', '[France|FRA]', 

2. Caption: Low-dose exposure in animals Page: Bisphenol A Section: Low-dose exposure in animals Columns: Dose (µg/kg/day), [Environmental_Working_Group|Environmental Working Group], Study Year Sample: ['0.025', '"Permanent changes to genital tract"', '2005']

3. Caption: Players Page: Charlotte Bobcats all-time roster Section: Players Columns: *, *, [2004_NBA_Expansion_Draft|2004 Expansion Draft], [2004_NBA_Expansion_Draft|2004 Expansion Draft], [2004_NBA_Expansion_Draft|2004 Expansion Draft], [2004_NBA_Expansion_


In [5]:
# Generate descriptions for all tables
print("Generating LLM descriptions for all tables...")
print("NOTE: This will make ~3K LLM API calls. Consider batch processing or caching.")
print()

table_descriptions = []
table_ids = []

for idx, row in tqdm(tables_df.iterrows(), total=len(tables_df)):
    table_descriptions.append(generate_table_description_with_llm(row))
    table_ids.append(row['table_id'])

print(f"Generated {len(table_descriptions)} descriptions")
print(f"Length stats - Mean: {np.mean([len(d) for d in table_descriptions]):.1f}, Max: {max([len(d) for d in table_descriptions])}")

Generating LLM descriptions for all tables...
NOTE: This will make ~3K LLM API calls. Consider batch processing or caching.



100%|██████████| 2932/2932 [00:00<00:00, 36674.41it/s]

Generated 2932 descriptions
Length stats - Mean: 224.0, Max: 256


## Encode Tables

In [6]:
# Load encoder
model_name = 'all-MiniLM-L6-v2'
print(f"Loading {model_name}...")
encoder = SentenceTransformer(model_name)
print(f"Dimension: {encoder.get_sentence_embedding_dimension()}")

Loading all-MiniLM-L6-v2...
Dimension: 384


In [7]:
# Encode descriptions
print("Encoding...")
table_embeddings = encoder.encode(
    table_descriptions, batch_size=32, show_progress_bar=True,
    convert_to_numpy=True, normalize_embeddings=True
)
print(f"Shape: {table_embeddings.shape}")

Encoding...


Batches: 100%|██████████| 92/92 [00:07<00:00, 11.67it/s]

Shape: (2932, 384)


## Build FAISS Index

In [8]:
# Build FAISS index
print("Building index...")
index = faiss.IndexFlatIP(encoder.get_sentence_embedding_dimension())
index.add(table_embeddings.astype('float32'))
print(f"✓ Index built with {index.ntotal} tables")

Building index...
✓ Index built with 2932 tables


### Test Custom Queries

In [9]:
import re

# Test custom queries - change test_query and run
test_query = "olympic medals table"
top_k = 5

print(f"Query: '{test_query}'")
print("=" * 80)

# ============================================================
# TODO: LLM CALL HERE - Generate hypothetical table description from query
# ============================================================
# Prompt: "Given the search query: '{test_query}', generate a description 
# of what a relevant table would contain. Describe the table structure, 
# columns, and type of data it would have. Keep it under 256 characters."
#
# Example LLM call:
# response = openai.ChatCompletion.create(
#     model="gpt-4",
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant."},
#         {"role": "user", "content": f"Given the search query: '{test_query}', generate a description of what a relevant table would contain. Describe the table structure, columns, and type of data it would have. Keep it under 256 characters."}
#     ],
#     max_tokens=100,
#     temperature=0.3
# )
# hypothetical_description = response.choices[0].message.content.strip()
# ============================================================

sleep_timer = 5
time.sleep(sleep_timer)

system_msg = 'You are a helpful assistant.'

user_msg = """
I will give you a keyword search query, and I want you to generate a description of an example table relevant to the query.
Follow the guidelines below when generating your description:
1. First generate a short description of the table, no more than 2 sentences.
2. Next generate a short caption for the table.
3. Then, suppose this table is on a website like wikipedia. Generate the title of the web page that this table would exist in.
4. Also, generate the section name of the web page that this table would exist in.
5. Generate the column names of this table.
6. Generate one row of example data. Ensure the data is of the appropriate data type.
7. Finally, output a JSON object with the following fields, 'description', 'table_caption', 'page_title', 'section_title', 'columns', 'sample_data'.

[Query]
world interest rates table

[Answer]

**Reasoning:**
I need to generate metadata associated with an example table relevant to the query. A table containing information about Eurozone interest rates would be highly relevant to this query.

**Description:**
A table summarizing historical Eurozone interest rates, showing—by date—the levels of the deposit facility rate, the main refinancing operations rate, and the marginal lending facility rate set by the European Central Bank.

**Table Caption:**
Interest rates

**Page Title:**
Eurozone

**Section Title:**
Interest rates

**Columns:**
['Date', 'Deposit facility', 'Main refinancing operations', 'Marginal lending facility']

**Sample Row:**
['1999-01-01', '2.00', '3.00', '4.50']

**JSON:**
{
  "description": "A table summarizing historical Eurozone interest rates, listing by date the European Central Bank’s key policy rates: the deposit facility rate, the main refinancing operations rate, and the marginal lending facility rate.",
  "table_caption": "Interest rates",
  "page_title": "Eurozone",
  "section_title": "Interest rates",
  "columns": [
    "Date",
    "Deposit facility",
    "Main refinancing operations",
    "Marginal lending facility"
  ],
  "sample_data": [
      "1999-01-01",
      "2.00",
      "3.00",
      "4.50"
  ]
}

"""

user_msg += f"""
[Query]
{test_query}

[Answer]

**Reasoning:**
"""

# Regex to extract a JSON object (handles quoted strings with escaped characters)
json_regex = re.compile(
    r'\{(?:[^{}"]|"[^"\\]*(?:\\.[^"\\]*)*")*\}',
    re.DOTALL
)

response = client.chat.completions.create(model="gpt-4o",
messages=[{"role": "system", "content": system_msg}, {"role": "user", "content": user_msg}],
max_tokens=4000, 
temperature=0.0)

hypothetical_description = response.choices[0].message.content

match = json_regex.search(hypothetical_description)

if not match:
  # Fallback: use query directly (replace with LLM-generated description)
  hypothetical_description = test_query
  print(f"Hypothetical table description: {hypothetical_description}\n")
else:
  json_str = match.group(0)

  # Parse the extracted JSON
  data = json.loads(json_str)

  metadata = []
  metadata.append(f"Caption: {data['table_caption']}")
  metadata.append(f"Page: {data['page_title']}")
  metadata.append(f"Section: {data['section_title']}")
  metadata.append(f"Columns: {data['columns']}")
  metadata.append(f"Sample: {data['sample_data']}")

  hypothetical_description = ' '.join(metadata)
  print(f"Hypothetical table description: {hypothetical_description}\n")

# Encode and search
test_emb = encoder.encode([hypothetical_description], convert_to_numpy=True, normalize_embeddings=True).astype('float32')
scores_test, indices_test = index.search(test_emb, top_k)

for rank in range(top_k):
    idx = indices_test[0][rank]
    table_id = table_ids[idx]
    row = tables_df[tables_df['table_id'] == table_id].iloc[0]
    
    print(f"\n{rank + 1}. {table_id} (score: {scores_test[0][rank]:.4f})")
    print(f"   Page: {row['page_title']}")
    print(f"   Caption: {row['table_caption']}")
    print(f"   Description: {table_descriptions[idx][:120]}...")
    print("-" * 80)

Query: 'olympic medals table'
Hypothetical table description: Caption: Olympic Medal Count Page: Summer Olympics 2024 Section: Medal Tally Columns: ['Country', 'Gold', 'Silver', 'Bronze', 'Total'] Sample: ['USA', 39, 41, 33, 113]


1. table-0407-338 (score: 0.8251)
   Page: Israel at the Paralympics
   Caption: Medal tables
   Description: Caption: Medal tables Page: Israel at the Paralympics Section: Medal tables Columns: Team (IOC code), № Summer, Gold, Br...
--------------------------------------------------------------------------------

2. table-0529-770 (score: 0.7449)
   Page: Lists of Olympic medalists
   Caption: Summer Olympic Games
   Description: Caption: Summer Olympic Games Page: Lists of Olympic medalists Section: Summer Olympic Games Columns: Games, Medal, Meda...
--------------------------------------------------------------------------------

3. table-0375-830 (score: 0.7427)
   Page: Ice hockey at the Olympic Games
   Caption: Medal winners
   Description: Caption: M

## Evaluation

### Load Queries and Relevance Judgments

In [10]:
# Load queries
queries = {}
with open('data/queries.txt', 'r') as f:
    for line in f:
        parts = line.strip().split(None, 1)
        if len(parts) == 2:
            query_id, query_text = parts
            queries[query_id] = query_text

print(f"Loaded {len(queries)} queries")
print("Examples:", list(queries.items())[:3])

# Load qrels (relevance judgments)
qrels = defaultdict(dict)
with open('data/qrels.txt', 'r') as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) >= 4:
            query_id, table_id, relevance = parts[0], parts[2], int(parts[3])
            qrels[query_id][table_id] = relevance

qrels = dict(qrels)
print(f"Loaded qrels for {len(qrels)} queries")

Loaded 60 queries
Examples: [('1', 'world interest rates table'), ('2', '2008 beijing olympics'), ('3', 'fast cars')]
Loaded qrels for 60 queries


---

### Generate Hypothetical Descriptions from Queries (HyDE)

In [ ]:
query_ids = list(queries.keys())
query_texts = [queries[qid] for qid in query_ids]

print(f"Generating hypothetical table descriptions for {len(query_texts)} queries...")
print("NOTE: This will make ~60 LLM API calls.")
print()

hypothetical_descriptions = []

for query_text in tqdm(query_texts):
    sleep_timer = 5
    time.sleep(sleep_timer)

    system_msg = 'You are a helpful assistant.'

    user_msg = """
    I will give you a keyword search query, and I want you to generate a description of an example table relevant to the query.
    Follow the guidelines below when generating your description:
    1. First generate a short description of the table, no more than 2 sentences.
    2. Next generate a short caption for the table.
    3. Then, suppose this table is on a website like wikipedia. Generate the title of the web page that this table would exist in.
    4. Also, generate the section name of the web page that this table would exist in.
    5. Generate the column names of this table.
    6. Generate one row of example data. Ensure the data is of the appropriate data type.
    7. Finally, output a JSON object with the following fields, 'description', 'table_caption', 'page_title', 'section_title', 'columns', 'sample_data'.

    [Query]
    world interest rates table

    [Answer]

    **Reasoning:**
    I need to generate metadata associated with an example table relevant to the query. A table containing information about Eurozone interest rates would be highly relevant to this query.

    **Description:**
    A table summarizing historical Eurozone interest rates, showing—by date—the levels of the deposit facility rate, the main refinancing operations rate, and the marginal lending facility rate set by the European Central Bank.

    **Table Caption:**
    Interest rates

    **Page Title:**
    Eurozone

    **Section Title:**
    Interest rates

    **Columns:**
    ['Date', 'Deposit facility', 'Main refinancing operations', 'Marginal lending facility']

    **Sample Row:**
    ['1999-01-01', '2.00', '3.00', '4.50']

    **JSON:**
    {
    "description": "A table summarizing historical Eurozone interest rates, listing by date the European Central Bank’s key policy rates: the deposit facility rate, the main refinancing operations rate, and the marginal lending facility rate.",
    "table_caption": "Interest rates",
    "page_title": "Eurozone",
    "section_title": "Interest rates",
    "columns": [
        "Date",
        "Deposit facility",
        "Main refinancing operations",
        "Marginal lending facility"
    ],
    "sample_data": [
        "1999-01-01",
        "2.00",
        "3.00",
        "4.50"
    ]
    }

    """

    user_msg += f"""
    [Query]
    {query_text}

    [Answer]

    **Reasoning:**
    """

    # Regex to extract a JSON object (handles quoted strings with escaped characters)
    json_regex = re.compile(
        r'\{(?:[^{}"]|"[^"\\]*(?:\\.[^"\\]*)*")*\}',
        re.DOTALL
    )

    response = client.chat.completions.create(model="gpt-4o",
    messages=[{"role": "system", "content": system_msg}, {"role": "user", "content": user_msg}],
    max_tokens=4000, 
    temperature=0.0)

    hypothetical_description = response.choices[0].message.content

    match = json_regex.search(hypothetical_description)

    if not match:
        # Fallback: use query directly (replace with LLM-generated description)
        hypothetical_description = query_text
        print(f"Hypothetical table description: {hypothetical_description}\n")
    else:
        json_str = match.group(0)

        # Parse the extracted JSON
        data = json.loads(json_str)

        metadata = []
        metadata.append(f"Caption: {data['table_caption']}")
        metadata.append(f"Page: {data['page_title']}")
        metadata.append(f"Section: {data['section_title']}")
        metadata.append(f"Columns: {data['columns']}")
        metadata.append(f"Sample: {data['sample_data']}")

        hypothetical_description = ' '.join(metadata)
        print(f"Hypothetical table description: {hypothetical_description}\n")

    hypothetical_descriptions.append(hypothetical_description)

print(f"Generated {len(hypothetical_descriptions)} hypothetical descriptions")
print(f"\nExamples:")
for i in range(3):
    print(f"  Query: {query_texts[i]}")
    print(f"  Hypothetical: {hypothetical_descriptions[i]}")
    print()

Generating hypothetical table descriptions for 60 queries...
NOTE: This will make ~60 LLM API calls.



  2%|▏         | 1/60 [00:08<08:20,  8.48s/it]

Hypothetical table description: Caption: Global Central Bank Interest Rates Page: World Interest Rates Section: Current Interest Rates by Country Columns: ['Country', 'Central Bank', 'Interest Rate (%)'] Sample: ['United States', 'Federal Reserve', '5.25']



  3%|▎         | 2/60 [00:16<07:54,  8.18s/it]

Hypothetical table description: Caption: 2008 Beijing Olympics Medal Tally Page: 2008 Beijing Olympics Section: Medal Tally Columns: ['Country', 'Gold', 'Silver', 'Bronze', 'Total'] Sample: ['China', 48, 22, 30, 100]



...

Hypothetical table description: Caption: Google Product Reviews Page: Google Products Section: User Reviews Columns: ['Product Name', 'Average Rating', 'Number of Reviews', 'Review Summary'] Sample: ['Google Pixel 6', '4.5', '1500', 'Users praise the camera quality and battery life.']



 98%|█████████▊| 59/60 [08:10<00:09,  9.47s/it]

Hypothetical table description: Caption: Closest Constellations to Earth Page: Constellations Section: Closest Constellations Columns: ['Constellation Name', 'Closest Star', 'Distance from Earth (light-years)'] Sample: ['Centaurus', 'Proxima Centauri', '4.24']



100%|██████████| 60/60 [08:19<00:00,  8.33s/it]

Hypothetical table description: Caption: Video Game Age Ratings Page: Video Game Ratings Section: Age Ratings Columns: ['Game Title', 'Release Date', 'Age Rating', 'Rating Board'] Sample: ['The Legend of Zelda: Breath of the Wild', '2017-03-03', 'E10+', 'ESRB']

Generated 60 hypothetical descriptions

Examples:
  Query: world interest rates table
  Hypothetical: Caption: Global Central Bank Interest Rates Page: World Interest Rates Section: Current Interest Rates by Country Columns: ['Country', 'Central Bank', 'Interest Rate (%)'] Sample: ['United States', 'Federal Reserve', '5.25']

  Query: 2008 beijing olympics
  Hypothetical: Caption: 2008 Beijing Olympics Medal Tally Page: 2008 Beijing Olympics Section: Medal Tally Columns: ['Country', 'Gold', 'Silver', 'Bronze', 'Total'] Sample: ['China', 48, 22, 30, 100]

  Query: fast cars
  Hypothetical: Caption: Fastest Production Cars Page: Fast Cars Section: Top Speed Records Columns: ['Car Model', 'Manufacturer', 'Top Speed (mph)', 'Year']

### Encode Hypothetical Descriptions and Retrieve

In [12]:
# Encode hypothetical descriptions (NOT raw queries)
print(f"Encoding {len(hypothetical_descriptions)} hypothetical descriptions...")
query_embeddings = encoder.encode(
    hypothetical_descriptions, batch_size=32, show_progress_bar=True,
    convert_to_numpy=True, normalize_embeddings=True
)

# Search top 100
k = 100
print(f"Searching top-{k}...")
scores, indices = index.search(query_embeddings.astype('float32'), k)

results = {qid: [table_ids[idx] for idx in indices[i]] for i, qid in enumerate(query_ids)}
print(f"✓ Retrieved {len(results)} query results")

# Show examples
print("\n" + "=" * 80)
print("EXAMPLE RESULTS (HyDE)")
print("=" * 80)
for qid in list(queries.keys())[:3]:
    print(f"\nQuery {qid}: '{queries[qid]}'")
    print(f"Hypothetical: {hypothetical_descriptions[query_ids.index(qid)][:80]}...")
    for rank, tid in enumerate(results[qid][:3], 1):
        rel = qrels.get(qid, {}).get(tid, 0)
        score_val = scores[query_ids.index(qid)][rank-1]
        page = tables_df[tables_df['table_id'] == tid].iloc[0]['page_title']
        print(f"  {rank}. {tid} (score: {score_val:.3f}, rel: {rel}) - {page}")
print("=" * 80)

Encoding 60 hypothetical descriptions...


Batches: 100%|██████████| 2/2 [00:00<00:00,  3.08it/s]

Searching top-100...
✓ Retrieved 60 query results

EXAMPLE RESULTS (HyDE)

Query 1: 'world interest rates table'
Hypothetical: Caption: Global Central Bank Interest Rates Page: World Interest Rates Section: ...
  1. table-0730-168 (score: 0.705, rel: 0) - List of Renminbi exchange rates
  2. table-0370-614 (score: 0.702, rel: 2) - Eurozone
  3. table-0552-599 (score: 0.673, rel: 0) - Rupee

Query 2: '2008 beijing olympics'
Hypothetical: Caption: 2008 Beijing Olympics Medal Tally Page: 2008 Beijing Olympics Section: ...
  1. table-0407-338 (score: 0.741, rel: 0) - Israel at the Paralympics
  2. table-0620-231 (score: 0.740, rel: 1) - Inna Zhukova
  3. table-0876-75 (score: 0.739, rel: 1) - Irina Risenzon

Query 3: 'fast cars'
Hypothetical: Caption: Fastest Production Cars Page: Fast Cars Section: Top Speed Records Colu...
  1. table-0990-862 (score: 0.739, rel: 1) - Speed Dreams
  2. table-1254-979 (score: 0.731, rel: 2) - Carbir Race Cars
  3. table-1275-187 (score: 0.678, rel: 2) - Pi

### Calculate Metrics

In [13]:
# Evaluation functions
def recall_at_k(retrieved, relevant, k):
    if len(relevant) == 0:
        return 0.0
    retrieved_at_k = set(retrieved[:k])
    return len(retrieved_at_k & relevant) / len(relevant)

def ndcg_at_k(retrieved, relevance, k):
    if len(relevance) == 0:
        return 0.0
    dcg = sum(relevance.get(retrieved[i], 0) / np.log2(i + 2) for i in range(min(k, len(retrieved))))
    ideal_rels = sorted(relevance.values(), reverse=True)[:k]
    idcg = sum(rel / np.log2(i + 2) for i, rel in enumerate(ideal_rels))
    return dcg / idcg if idcg > 0 else 0.0

In [14]:
# Evaluate
k_values = [1, 5, 10, 20]
metrics = defaultdict(list)

for query_id, retrieved in results.items():
    if query_id not in qrels:
        continue
    relevance = qrels[query_id]
    relevant = set(tid for tid, rel in relevance.items() if rel > 0)
    
    for k in k_values:
        metrics[f'Recall@{k}'].append(recall_at_k(retrieved, relevant, k))
        metrics[f'nDCG@{k}'].append(ndcg_at_k(retrieved, relevance, k))

# Print results
print("\n" + "="*60)
print("HYDE EVALUATION RESULTS")
print("="*60)
print("\nRecall:")
for k in k_values:
    print(f"  Recall@{k:2d}: {np.mean(metrics[f'Recall@{k}']):.4f}")
print("\nnDCG:")
for k in k_values:
    print(f"  nDCG@{k:2d}  : {np.mean(metrics[f'nDCG@{k}']):.4f}")
print("="*60)


HYDE EVALUATION RESULTS

Recall:
  Recall@ 1: 0.1071
  Recall@ 5: 0.2647
  Recall@10: 0.3811
  Recall@20: 0.5166

nDCG:
  nDCG@ 1  : 0.5167
  nDCG@ 5  : 0.5187
  nDCG@10  : 0.5200
  nDCG@20  : 0.5427


## Inspect Results

In [15]:
# Inspect specific query results
query_id = '1'

print(f"Query {query_id}: {queries[query_id]}")
print(f"Hypothetical description: {hypothetical_descriptions[query_ids.index(query_id)]}")
print("="*80)

for i, table_id in enumerate(results[query_id][:5], 1):
    row = tables_df[tables_df['table_id'] == table_id].iloc[0]
    rel = qrels.get(query_id, {}).get(table_id, 0)
    score_val = scores[query_ids.index(query_id)][i-1]
    
    print(f"\n{i}. {table_id} (score: {score_val:.4f}, relevance: {rel})")
    print(f"   Page: {row['page_title']}")
    print(f"   Section: {row['section_title']}")
    print(f"   Caption: {row['table_caption']}")
    
    try:
        headers = json.loads(row['headers'])
        print(f"   Headers: {headers[:5]}{'...' if len(headers) > 5 else ''}")
    except:
        pass
    
    try:
        sample = json.loads(row['sample_data'])
        print(f"   Sample ({len(sample)} rows): {sample[0][:3]}...")
    except:
        pass
    
    print(f"   LLM Description: {table_descriptions[table_ids.index(table_id)]}")
    print("-" * 80)

Query 1: world interest rates table
Hypothetical description: Caption: Global Central Bank Interest Rates Page: World Interest Rates Section: Current Interest Rates by Country Columns: ['Country', 'Central Bank', 'Interest Rate (%)'] Sample: ['United States', 'Federal Reserve', '5.25']

1. table-0730-168 (score: 0.7047, relevance: 0)
   Page: List of Renminbi exchange rates
   Section: List of World Bank nominal exchange rates
   Caption: List of World Bank nominal exchange rates
   Headers: ['World Bank annual average middle exchange rate for US dollar to Chinese yuan ( 1 US dollar to Chinese yuan )', 'World Bank annual average middle exchange rate for US dollar to Chinese yuan ( 1 US dollar to Chinese yuan )', 'World Bank annual average middle exchange rate for US dollar to Chinese yuan ( 1 US dollar to Chinese yuan )', 'World Bank annual average middle exchange rate for US dollar to Chinese yuan ( 1 US dollar to Chinese yuan )', 'World Bank annual average middle exchange rate for US